# Traveling Salesman Problem
The Traveling Salesman Problem (TSP) is one of the most famous combinatorial optimization problems. 
The TSP goal is to find the shortest possible route that visits each city once and returns to the original city.
It is classified as an NP-hard problem in the field of combinatorial optimization.

## Problem Description
The TSP can be defined as follows: for a given list of cities and the distances between each pair of them, we want to find the shortest possible route that goes to each city once and returns to the origin city.

There is a class of Traveling Salesman Problems that assumes that the distance of going from city $$i$$ to city $j$  is the same as going form city $j$ to city $i$, this type of Traveling Salesman Problem  is also known as the symmetric Traveling Salesman Problem. In this example, we use Euclidean distances, but the TSP model formulation is valid independent of the way in which the individual distances are determined.


## Solution Approach

Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex decision problem. The Gurobi Optimizer solves such models using state-of-the-art mathematics and computer science.

A mathematical optimization model has five components, namely:

* Sets and indices.
* Parameters.
* Decision variables.
* Objective function(s).
* Constraints.

We now present a MIP formulation of the TSP that identifies the shortest route that goes to all the cities once and returns to the origin city.


## Python Implementation

Consider a salesperson that needs to visit customers at each state capital of the continental US. The salesperson wants to identify the shortest route that goes to all the state capitals.

This modeling example requires the following libraries that are not part of the standard Python distribution:
* **manganite**: to create the dashboard
* **folium**: to create maps.
* **gurobipy**: provides Gurobi algorithms to solve MIP models.


In [7]:
import manganite
%load_ext manganite

The manganite extension is already loaded. To reload it, use:
  %reload_ext manganite


In [8]:
import json
import math
from itertools import combinations
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import plotly.express as px



In [9]:
capitals_json = json.load(open('capitals.json'))

In [10]:
region_list = [ 'All', 'West', 'Pacific']

In [11]:
%%mnn widget --type radio region_list --tab "Capitals" --header "Region" --var r
r = region_list[0]

In [12]:
%%mnn widget --type table --tab "Capitals" --position 1 0 6 --header "DataFrame" --var df_cap

capitals = []
coordinates_lat = []
coordinates_long = []
if r == 'All':
    for state in capitals_json:
        if state not in ['AK', 'HI']:
            capital = capitals_json[state]['capital']
            capitals.append(capital)
            coordinates_lat.append(float(capitals_json[state]['lat']))
            coordinates_long.append(float(capitals_json[state]['long']))
elif r == 'West':
    west_states = ['AZ','CA','CO','ID' , 'MT', 'NV', 'NM', 'OR', 'UT', 'WA', 'WY']
    for state in capitals_json:
        if state in west_states:
            capital = capitals_json[state]['capital']
            capitals.append(capital)
            coordinates_lat.append(float(capitals_json[state]['lat']))
            coordinates_long.append(float(capitals_json[state]['long']))

elif r == 'Pacific':
    west_states = ['AZ','CA','CO','ID' , 'MT', 'NV', 'NM', 'OR', 'UT', 'WA', 'WY']
    for state in capitals_json:
        if state not in (west_states +  ['AK', 'HI']):
            capital = capitals_json[state]['capital']
            capitals.append(capital)
            coordinates_lat.append(float(capitals_json[state]['lat']))
            coordinates_long.append(float(capitals_json[state]['long']))



df_cap = pd.DataFrame({'Capital' : capitals, 'coordinates_lat' : coordinates_lat, 'coordinates_long': coordinates_long})

capitals = df_cap['Capital'].values.tolist()
coordinates = {}

for index, row in df_cap.iterrows():
    coordinates[row['Capital']] = (row['coordinates_lat'], row['coordinates_long'])



In [13]:


# Compute pairwise distance matrix

def distance(city1, city2):
    c1 = coordinates[city1]
    c2 = coordinates[city2]
    diff = (c1[0]-c2[0], c1[1]-c2[1])
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])


In [14]:
# Callback - use lazy constraints to eliminate sub-tours

def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < len(capitals):
            # add subtour elimination constr. for every pair of cities in subtour
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

# Given a tuplelist of edges, find the shortest subtour

def subtour(edges):
    unvisited = capitals[:]
    cycle = capitals[:] # Dummy - guaranteed to be replaced
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

In [15]:
%%mnn execute --on button "Optimize routes" --returns tour_df

dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(capitals, 2)}

# tested with Python 3.7 & Gurobi 9.0.0

m = gp.Model()

# Variables: is city 'i' adjacent to city 'j' on the tour?
vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

# Symmetric direction: Copy the object
for i, j in vars.keys():
    vars[j, i] = vars[i, j]  # edge in opposite direction

# Constraints: two edges incident to each city
cons = m.addConstrs(vars.sum(c, '*') == 2 for c in capitals)

m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

# Retrieve solution

vals = m.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == len(capitals)

tour_lat = []
tour_lon = []
for city in tour:
    tour_lat.append(coordinates[city][0])
    tour_lon.append(coordinates[city][1])

tour_df = pd.DataFrame({'Capital': tour, 'lat': tour_lat, 'lon':tour_lon})  

print('Optimal solution found')

Restricted license - for non-production use only - expires 2024-10-28
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 48 rows, 1128 columns and 2256 nonzeros
Model fingerprint: 0x63641a38
Variable types: 0 continuous, 1128 integer (1128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 48 rows, 1128 columns, 2256 nonzeros
Variable types: 0 continuous, 1128 integer (1128 binary)

Root relaxation: objective 1.611858e+02, 72 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  161.18576    0   12          -  161.1857

In [16]:
%%mnn widget --type plot --var fig_map --tab "Optimal Route"  --header "Solution map" --position 0 0 6

fig_map = px.line_mapbox(tour_df, lat="lat", lon="lon", zoom=3,  hover_name = 'Capital')

fig_map.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=3, mapbox_center_lat = 41,
    margin={"r":0,"t":0,"l":0,"b":0})

## Conclusions

The Traveling Salesman Problem (TSP) is the most popular combinatorial optimization problem. This problem is very easy to explain, although it is very complicated to solve. The largest TSP problem solved has 85,900 cities. The TSP is a source of discovery for new approaches to solve complex combinatorial optimization problems and has led to many applications.

In this modeling example, we have shown how to formulate the symmetric Traveling Salesman Problem as a MIP problem. We also showed how to dynamically eliminate subtours by using lazy constraints. 

## References

[1] D. L. Applegate, R. E. Bixby, V. Chvatal and W. J. Cook , The Traveling Salesman Problem: A Computational Study, Princeton University Press, Princeton, 2006.

[2] http://www.math.uwaterloo.ca/tsp/index.html

[3] https://www.youtube.com/watch?v=q8nQTNvCrjE&t=35s

[4] http://www.math.uwaterloo.ca/tsp/concorde.html

Copyright © 2020 Gurobi Optimization, LLC